In [1]:
#Library for environments
import gym
from gym.envs.registration import register
from gym.envs.toy_text.frozen_lake import generate_random_map

#Librairies to represent the output
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt

#Essential libraries for computation
import numpy as np
import random
from collections import deque

import pandas as pd


#Library for environments
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense

import tqdm

In [3]:
# TensorFlow config to GPU
import tensorflow as tf
print(tf.__version__)

gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=17292)] # 4139778048 , 17083400192, 16034824192
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(logical_gpus)
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")


from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

print()
print()

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2.6.0
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
1 Physical GPU, 1 Logical GPUs
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1341930220950718647
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 18131976192
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9333891021328153574
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


Num GPUs Available:  1


In [4]:
random_map = generate_random_map(size=10, p=0.3)
env = gym.make("FrozenLake-v1", desc=random_map, is_slippery=True)
env.reset()
env.render()


SFFFHHHHFH
FFHFHHHFHH
HHFFFFHFHH
HHFFHFFHHH
HHHHHFFHHH
HFFHHHFHHF
HHFFFHFFFH
HHHHHHFHHF
HHHFHHFHHH
HHFHFHFFFG


In [5]:
discount_factor = 0.95
eps = 0.5
eps_decay_factor = 0.999
num_episodes=100000

In [6]:
model = Sequential()
model.add(InputLayer(batch_input_shape=(1, env.observation_space.n)))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(env.action_space.n, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (1, 20)                   2020      
_________________________________________________________________
dense_1 (Dense)              (1, 20)                   420       
_________________________________________________________________
dense_2 (Dense)              (1, 20)                   420       
_________________________________________________________________
dense_3 (Dense)              (1, 4)                    84        
Total params: 2,944
Trainable params: 2,944
Non-trainable params: 0
_________________________________________________________________


In [9]:
# tensorflow in gpu
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [10]:
for i in tqdm.tqdm(range(num_episodes)):
    state = env.reset()
    eps *= eps_decay_factor
    done = False
    while not done:
        if np.random.random() < eps:
            action = np.random.randint(0, env.action_space.n)
        else:
            action = np.argmax(
              model.predict(np.identity(env.observation_space.n)[state:state + 1]))
        new_state, reward, done, _ = env.step(action)
        target = reward + discount_factor * np.max(model.predict(np.identity(env.observation_space.n)[new_state:new_state + 1]))
        target_vector = model.predict(
          np.identity(env.observation_space.n)[state:state + 1])[0]
        target_vector[action] = target
        history = model.fit(
          np.identity(env.observation_space.n)[state:state + 1], 
          target_vector.reshape(-1, env.action_space.n), 
          epochs=1, verbose=0)
        state = new_state
    clear_output(wait=True)

100%|██████████| 100000/100000 [23:51:50<00:00,  1.16it/s]


In [15]:
num_episodes

100000

In [20]:
# Evaluate agent's performance after Q-learning
while reward != 1.0:
    state = env.reset()
    done = False
    while not done:
        action = np.argmax(model.predict(np.identity(env.observation_space.n)[state:state + 1]))
        state, reward, done, _ = env.step(action)
        clear_output(wait=True)
        env.render()
        time.sleep(1)

  (Down)
SFFFHHHHFH
FFHFHHHFHH
HHFFFFHFHH
HHFFHFFHHH
HHHHHFFHHH
HFFHHHFHHF
HHFFFHFFFH
HHHHHHFHHF
HHHFHHFHHH
HHFHFHFFFG


KeyboardInterrupt: 

In [11]:
# accuracy of the model
scores = model.evaluate(np.identity(env.observation_space.n), model.predict(np.identity(env.observation_space.n)))
# print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

4/4 [==============================] - 2s 2ms/step - loss: 0.0000e+00 - mae: 0.0000e+00


In [21]:
model.predict(np.identity(env.observation_space.n))

array([[-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.5910792e-03, -3.7148778e-04, -7.4915369e-03, -5.5835787e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e-03, -5.7311011e-03],
       [-8.9611961e-03, -4.4778593e-34, -8.2201520e

In [ ]:
# plot
import matplotlib.pyplot as plt
plt.plot(history.history['mae'])
plt.show()

In [ ]:
print(history.history)